In [17]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

import sys
sys.path.append('../../scripts')

from btsc_dataset import load_full_dataset, split_images_and_labels, resize_images
from utils import train_validation_test_split

import numpy as np

In [18]:
resized_data = resize_images(load_full_dataset(), size=32)
(images, labels) = split_images_and_labels(resized_data)
unique_labels = set(labels)

print('Images:', images.shape)
print('Labels:', labels.shape)
print('Total Unique Labels:', len(unique_labels))

Images: (7095, 32, 32, 3)
Labels: (7095,)
Total Unique Labels: 62


In [19]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

integer_labels= LabelEncoder().fit_transform(labels)
ont_hot_labels = to_categorical(integer_labels)

X = images
y = ont_hot_labels

X_train, X_valid, X_test, \
y_train, y_valid, y_test = train_validation_test_split(X, y, random_state=42)

print('X:', X.shape)
print('y:', y.shape)
print()
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print()
print('X_valid:', X_valid.shape)
print('y_valid:', y_valid.shape)
print()
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

X: (7095, 32, 32, 3)
y: (7095, 62)

X_train: (4257, 32, 32, 3)
y_train: (4257, 62)

X_valid: (1419, 32, 32, 3)
y_valid: (1419, 62)

X_test: (1419, 32, 32, 3)
y_test: (1419, 62)


In [ ]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.applications import *

vgg_model = VGG19(
  weights='imagenet',
  include_top=False,
  input_shape=X[0].shape,
)

layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

x = layer_dict['block2_pool'].output

x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dense(len(unique_labels), activation='softmax')(x)

model = Model(inputs=vgg_model.input, outputs=x)

for layer in model.layers[0:7]:
    layer.trainable = False

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'],
)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=0, 
    mode='min',
)
model_checkpoint = ModelCheckpoint(
    'vgg19_final_weights.hdf5', 
    save_best_only=True, 
    save_weights_only=True,
    monitor='val_loss', 
    mode='min',
)
reduce_lr_loss = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1, 
    patience=7, 
    verbose=1, 
    min_delta=1e-4, 
    mode='min',
)

history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_valid, y_valid), 
    batch_size=50, 
    epochs=50, 
    verbose=1,
    callbacks=[early_stopping, model_checkpoint, reduce_lr_loss], 
)

model.load_weights('vgg19_final_weights.hdf5')

Train on 4257 samples, validate on 1419 samples
Epoch 1/50
4257/4257 [==============================] - 22s 5ms/step - loss: 2.2022 - acc: 0.5924 - val_loss: 0.8523 - val_acc: 0.8069
Epoch 2/50
4257/4257 [==============================] - 22s 5ms/step - loss: 0.5470 - acc: 0.8825 - val_loss: 0.4171 - val_acc: 0.9084
Epoch 3/50
4257/4257 [==============================] - 20s 5ms/step - loss: 0.2594 - acc: 0.9361 - val_loss: 0.2500 - val_acc: 0.9401
Epoch 4/50
4257/4257 [==============================] - 20s 5ms/step - loss: 0.1090 - acc: 0.9732 - val_loss: 0.2095 - val_acc: 0.9563
Epoch 5/50
4257/4257 [==============================] - 20s 5ms/step - loss: 0.0885 - acc: 0.9770 - val_loss: 0.2207 - val_acc: 0.9493
Epoch 6/50
4257/4257 [==============================] - 20s 5ms/step - loss: 0.0303 - acc: 0.9939 - val_loss: 0.1427 - val_acc: 0.9676
Epoch 7/50
4257/4257 [==============================] - 21s 5ms/step - loss: 0.0167 - acc: 0.9972 - val_loss: 0.1547 - val_acc: 0.9626
Epoch 8

In [ ]:
# Plot training & validation accuracies per epoch
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
# Plot training & validation losses per epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)

print('Test Results:')
print('  Loss     =', loss)
print('  Accuracy =', acc)